In [2]:
import pandas as pd
import numpy as np

In [3]:
df=pd.read_csv('train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
# checking null
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [5]:
# number of rows and column
df.shape

(20800, 5)

In [6]:
# more information about dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [7]:
# dropping the null values
df=df.dropna()

In [8]:
# Get independent features

X=df.drop('label',axis=1)

In [9]:
# Get dependent Features
y=df['label']

In [10]:
X.shape

(18285, 4)

In [11]:
y.shape

(18285,)

In [12]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [13]:
# Vocabulary size
voc_size=5000

## One Hot repersentation

In [14]:
message=X.copy()

In [15]:
message

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [16]:
message.reset_index(inplace=True)

In [17]:
import nltk
import re
from nltk.corpus import stopwords

In [18]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(message)):
    review = re.sub('[^a-zA-Z]', ' ', message['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [20]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [21]:
oneHot_reper=[one_hot(word,voc_size) for word in corpus]
oneHot_reper

[[1563, 1142, 2300, 4089, 4635, 681, 2696, 3456, 2758, 3881],
 [3138, 2499, 4324, 4593, 835, 468, 2845],
 [2618, 647, 3770, 1995],
 [1561, 1769, 4266, 1423, 3364, 1707],
 [1691, 835, 1208, 4854, 1815, 2319, 835, 1492, 4225, 1238],
 [468,
  4470,
  3981,
  3606,
  4311,
  322,
  3944,
  1106,
  1353,
  2409,
  1390,
  2192,
  3434,
  2157,
  2845],
 [2944, 2825, 3786, 467, 3631, 2597, 3782, 4716, 1429, 2566, 1801],
 [4594, 3560, 4864, 3646, 4503, 3229, 322, 4947, 1429, 2566, 1801],
 [4156, 3107, 1968, 3787, 2884, 3916, 4229, 1068, 322, 2150],
 [2940, 3516, 2766, 1479, 1177, 4366, 4215, 1115],
 [2394, 2735, 2133, 1113, 1627, 2116, 2441, 210, 3887, 343, 1904],
 [1423, 3184, 4635, 3916, 322, 4503],
 [3746, 900, 405, 3262, 205, 1690, 4892, 3001, 4084],
 [3317, 482, 3096, 3165, 2086, 3781, 545, 1429, 2566, 1801],
 [3580, 3525, 387, 4651, 3307, 1429, 2566, 1801],
 [4357, 4063, 2409, 2208, 1981, 1490, 433, 3242, 1589, 1226],
 [3840, 3187, 2499],
 [103, 1395, 3726, 1450, 322, 1854, 2229, 2845],

## Word Embedding


In [22]:

# pre padding
sent_legth=20

embedded_sent=pad_sequences(oneHot_reper,padding='pre',maxlen=sent_legth)
embedded_sent

array([[   0,    0,    0, ..., 3456, 2758, 3881],
       [   0,    0,    0, ...,  835,  468, 2845],
       [   0,    0,    0, ...,  647, 3770, 1995],
       ...,
       [   0,    0,    0, ..., 1429, 2566, 1801],
       [   0,    0,    0, ..., 1162, 2932, 3815],
       [   0,    0,    0, ..., 1970, 3572, 4106]])

## Creating Model

In [23]:
embedding_feature_vec=40
model=Sequential()
model.add(Embedding(input_dim=voc_size,output_dim=embedding_feature_vec,input_shape=(sent_legth,)))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

C:\Users\ACER\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:81: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 20, 40)              │         200,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 100)                 │          56,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             101 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 256,501 (1001.96 KB)

 Trainable params: 256,501 (1001.96 KB)

 Non-trainable params: 0 (0.00 B)

None


In [24]:
len(embedded_sent)

18285

In [25]:
X_final=np.array(embedded_sent)
y_final=np.array(y)

In [26]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [27]:
from sklearn.model_selection import train_test_split


In [28]:
X_train,X_test,y_train,y_test=train_test_split(X_final,y_final,test_size=0.2,random_state=2)

## Model Training

In [29]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 9s 20ms/step - accuracy: 0.7988 - loss: 0.4080 - val_accuracy: 0.9144 - val_loss: 0.1942
Epoch 2/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.9380 - loss: 0.1535 - val_accuracy: 0.9259 - val_loss: 0.1996
Epoch 3/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.9568 - loss: 0.1081 - val_accuracy: 0.9207 - val_loss: 0.2028
Epoch 4/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.9674 - loss: 0.0869 - val_accuracy: 0.9180 - val_loss: 0.2181
Epoch 5/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.9761 - loss: 0.0664 - val_accuracy: 0.9204 - val_loss: 0.2121
Epoch 6/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.9846 - loss: 0.0501 - val_accuracy: 0.9226 - val_loss: 0.2639
Epoch 7/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.9910 - loss: 0.0336 - val_accuracy: 0.9182 - val_loss: 0.3018
Epoch 8/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.9934 - loss: 0.0225 - val_accu

## Model Prediction
`

In [30]:
y_pred=model.predict(X_test)

115/115 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step


In [31]:
y_pred=np.where(y_pred>0.5,1,0)

In [32]:
from sklearn.metrics import confusion_matrix,accuracy_score

In [33]:
confusion_matrix(y_test,y_pred)

array([[1859,  192],
       [  94, 1512]], dtype=int64)

In [34]:
accuracy_score(y_test,y_pred)

0.9217938200710966

### Predictive System

In [35]:
sent=['Teen â€™Geisha Dollsâ€™ Gang Busted for Armed Robberies - Breitbart']
oneHot_sent=[one_hot(word,voc_size) for word in sent]
embedded_sent=pad_sequences(oneHot_sent,padding='pre',maxlen=20)
embedded_sent

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
        4330, 1363,  715, 3171, 3895, 3774, 2901, 4420, 2845]])

In [36]:
prediction=model.predict(embedded_sent)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


In [37]:
prediction=np.where(prediction >0.5,1,0)

In [ ]:
prediction